Downloading Model: https://medium.com/@karankakwani/build-and-run-llama2-llm-locally-a3b393c1570e


Load Model From: https://python.langchain.com/v0.1/docs/guides/development/local_llms/

# Set-up and Imports

In [42]:
%env CMAKE_ARGS="-DLLAMA_METAL=on"
%env FORCE_CMAKE=1
%pip install --upgrade --quiet  llama-cpp-python --no-cache-dirclear

env: CMAKE_ARGS="-DLLAMA_METAL=on"
env: FORCE_CMAKE=1

Usage:   
  /Users/k2370941/Desktop/llama2-7B/.venv/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /Users/k2370941/Desktop/llama2-7B/.venv/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /Users/k2370941/Desktop/llama2-7B/.venv/bin/python -m pip install [options] [-e] <vcs project url> ...
  /Users/k2370941/Desktop/llama2-7B/.venv/bin/python -m pip install [options] [-e] <local project path> ...
  /Users/k2370941/Desktop/llama2-7B/.venv/bin/python -m pip install [options] <archive url/path> ...

no such option: --no-cache-dirclear
Note: you may need to restart the kernel to use updated packages.


In [14]:
!pip install --quiet llama-cpp-python

In [80]:
# Import packages and load model

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path="/Users/k2370941/Desktop/llama2-7B/llama.cpp/models/7B/ggml-model-q4_0.bin", # path to model file - .bin file contains the weights of the model, which have been quantized to reduce their size and improve inference speed.
    n_gpu_layers=1, # specifies that one layer of the model should be executed on the GPU, which can help improve performance
    n_batch=512, # determines number of tokens to process in a single batch - larger batch size can improve throughput but requires more memory
    n_ctx=2048,# sets the maximum number of tokens the model can consider in its context - Increasing this value allows the model to handle longer inputs but requires more memory
    f16_kv=True, # enables the use of 16-bit floating-point precision for key-value caches, which can reduce memory usage and potentially increase speed without significantly affecting model performance
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), # used to stream output as the model processes data - Callbacks can be used for logging, monitoring, or modifying the behavior of the model during inference
    verbose=True # more detailed outputs during processing, which can be useful for debugging or understanding the model's behaviour
)

llama_model_loader: loaded meta data with 17 key-value pairs and 291 tensors from /Users/k2370941/Desktop/llama2-7B/llama.cpp/models/7B/ggml-model-q4_0.bin (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 32000
llama_model_loader: - kv   3:                       llama.context_length u32              = 4096
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   7:      

In [44]:
# Import more packages

from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain_core.prompts import PromptTemplate

from langchain.prompts.few_shot import FewShotPromptTemplate

# Experiment 1: Span Extraction using few-shot learning

In [82]:
# generate prompts for extracting specific spans of text from a given input

example_prompt = PromptTemplate(input_variables=["Text", "Spans"], 
                template="You are a professional assistant for extracting span from text given to you. Text fragment for span extraction is: {Text}\n Extracted Spans:{Spans}")

In [83]:
examples = [
  {
    "Text": "she lost her job and was feeling very low. She couldn't take care of herself and looked unkempt",
    "Spans": 
"""
- "lost her job"
- "couldn't take care of herself"
- "looked unkempt"
"""
  },
  {
    "Text": "he travelled by himself and looked well kempt, he was in an office job and enjoying it",
    "Spans": 
"""
- "travelled by himself"
- "looked well kempt"
- "was in an office job"
"""
  }
]


In [84]:
# format prompt using first example

print(example_prompt.format(**examples[0]))

You are a professional assistant for extracting span from text given to you. Text fragment for span extraction is: she lost her job and was feeling very low. She couldn't take care of herself and looked unkempt
 Extracted Spans:
- "lost her job"
- "couldn't take care of herself"
- "looked unkempt"



In [85]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="You are a professional assistant for extracting spans from text fragments given to you. Text fragment for spans extraction is: {input}",
    input_variables=["input"]
)

In [86]:
# include special tags <<SYS>> for system message, [INST] for instruction

DEFAULT_LLAMA_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""<<SYS>> \n You are a professional assistant for extracting spans from text fragments given to \
you. \n <</SYS>> \n\n [INST] Generate text fragment for spans extraction is: \n\n {input} [/INST]""",
)

# without special tags
DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are a professional assistant for extracting spans from text fragments given to \
you. Generate text fragment for spans extraction is: {input}""",
)

In [87]:
QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[(lambda llm: isinstance(llm, LlamaCpp), DEFAULT_LLAMA_SEARCH_PROMPT)],
)

In [88]:
prompt = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt

PromptTemplate(input_variables=['input'], template='<<SYS>> \n You are a professional assistant for extracting spans from text fragments given to you. \n <</SYS>> \n\n [INST] Generate text fragment for spans extraction is: \n\n {input} [/INST]')

In [89]:
# Chain
llm_chain = LLMChain(prompt=few_shot_prompt, llm=llm)
question = "she was doing quite well in her job and appeared well kempt. She presented herself well and was also offered a promotion."
llm_chain.run({"input": question})


 Extracted Spans:
- "doing quite well"
- "appeared well kempt"
- "was also offered a promotion"



llama_print_timings:        load time =    4483.14 ms
llama_print_timings:      sample time =       2.80 ms /    35 runs   (    0.08 ms per token, 12482.17 tokens per second)
llama_print_timings: prompt eval time =   25494.09 ms /   434 tokens (   58.74 ms per token,    17.02 tokens per second)
llama_print_timings:        eval time =    1650.60 ms /    34 runs   (   48.55 ms per token,    20.60 tokens per second)
llama_print_timings:       total time =    6193.97 ms /   468 tokens


'\n Extracted Spans:\n- "doing quite well"\n- "appeared well kempt"\n- "was also offered a promotion"\n'

# Experiment 2: Medication Extraction using zero-shot and few-shot learning

In [90]:
# Example document
new_clinical_document = """
The patient was initially prescribed Metoprolol 50 mg twice daily for hypertension. 
During the first follow-up visit, the dosage of Metoprolol was increased to 100 mg twice daily. 
Later, the patient developed side effects and Metoprolol was switched to Atenolol 50 mg once daily. 
In the subsequent visit, Amlodipine 5 mg was added to the treatment plan. 
At the final follow-up, the Atenolol dosage was increased to 100 mg once daily, and Amlodipine was continued.
"""

In [91]:
# zero-shot
def zero_shot_medication_extraction(text):
    prompt = f"Extract all medications and their doses from the following clinical document:\n\n{text}"
    response = llm(prompt)
    return response


In [93]:
zero_shot_extraction_result = zero_shot_medication_extraction(new_clinical_document)
print("Zero-shot medication extraction result:\n", zero_shot_extraction_result)

Llama.generate: prefix-match hit



### Code List

| Name | Description |
|----|----|
|Atenolol |The medicine used for blood pressure control. |
|Metoprolol |The medicine used for blood pressure control. |
|Amlodipine |Used in treatment of angina or hypertension, as well as prevention of heart attacks and strokes. It is also used to treat Raynaud's disease (a condition that causes some areas of your body, such as your fingers and toes, to feel cold). |
|Hypertension |High blood pressure |
|Angina pectoris |Chest pain caused by reduced oxygen supply to the heart due to narrowed arteries in your heart. Angina is a symptom of coronary artery disease that occurs when your heart isn't getting enough oxygen-rich blood. It can be a dull, pressure-like or squeezing sensation in your chest. |
|Myocardial infarction |The medical term for a heart attack. A myocardial infarction is the death of a part of your heart muscle caused by blocked blood flow to the heart.


llama_print_timings:        load time =    4483.14 ms
llama_print_timings:      sample time =      19.69 ms /   256 runs   (    0.08 ms per token, 13003.50 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (     nan ms per token,      nan tokens per second)
llama_print_timings:        eval time =   13017.38 ms /   256 runs   (   50.85 ms per token,    19.67 tokens per second)
llama_print_timings:       total time =   13480.96 ms /   256 tokens


Zero-shot medication extraction result:
 
### Code List

| Name | Description |
|----|----|
|Atenolol |The medicine used for blood pressure control. |
|Metoprolol |The medicine used for blood pressure control. |
|Amlodipine |Used in treatment of angina or hypertension, as well as prevention of heart attacks and strokes. It is also used to treat Raynaud's disease (a condition that causes some areas of your body, such as your fingers and toes, to feel cold). |
|Hypertension |High blood pressure |
|Angina pectoris |Chest pain caused by reduced oxygen supply to the heart due to narrowed arteries in your heart. Angina is a symptom of coronary artery disease that occurs when your heart isn't getting enough oxygen-rich blood. It can be a dull, pressure-like or squeezing sensation in your chest. |
|Myocardial infarction |The medical term for a heart attack. A myocardial infarction is the death of a part of your heart muscle caused by blocked blood flow to the heart.


In [94]:
# few-shot
def few_shot_medication_extraction(text, examples):
    example_prompts = "\n\n".join([f"Text: {ex['text']}\nExtracted Medications: {ex['extracted_medications']}" for ex in examples])
    prompt = f"Use the following examples to extract medications and their doses from the given clinical document.\n\nExamples:\n{example_prompts}\n\nText: {text}\nExtracted Medications:"
    response = llm(prompt)
    return response

In [95]:
# Few-shot examples for medication extraction
extraction_examples = [
    {
        "text": "The patient was prescribed Metformin 500 mg twice daily for diabetes. After two months, the dose was increased to 1000 mg twice daily.",
        "extracted_medications": "[Metformin 500 mg twice daily], [Metformin 1000 mg twice daily]"
    },
    {
        "text": "Initially, Atorvastatin 20 mg was prescribed. During the first follow-up, the dosage was increased to 40 mg. Later, Ezetimibe 10 mg was added.",
        "extracted_medications": "[Atorvastatin 20 mg], [Atorvastatin 40 mg], [Ezetimibe 10 mg]"
    }
]

In [96]:
# Few-shot medication extraction
few_shot_extraction_result = few_shot_medication_extraction(new_clinical_document, extraction_examples)
print("Few-shot medication extraction result:\n", few_shot_extraction_result)

Llama.generate: prefix-match hit


 [Metoprolol 50 mg twice daily], [Atenolol 50 mg once daily], [Atenolol 100 mg once daily], [Amlodipine 5 mg]



llama_print_timings:        load time =    4483.14 ms
llama_print_timings:      sample time =       4.40 ms /    53 runs   (    0.08 ms per token, 12042.72 tokens per second)
llama_print_timings: prompt eval time =    5154.48 ms /   355 tokens (   14.52 ms per token,    68.87 tokens per second)
llama_print_timings:        eval time =    2563.29 ms /    52 runs   (   49.29 ms per token,    20.29 tokens per second)
llama_print_timings:       total time =    7814.68 ms /   407 tokens


Few-shot medication extraction result:
  [Metoprolol 50 mg twice daily], [Atenolol 50 mg once daily], [Atenolol 100 mg once daily], [Amlodipine 5 mg]



# Experiment 3: Medication Annotation using zero-shot and few-shot learning

In [97]:
# Example  document
new_clinical_document = """
The patient was initially prescribed Metoprolol 50 mg twice daily for hypertension. 
During the first follow-up visit, the dosage of Metoprolol was increased to 100 mg twice daily. 
Later, the patient developed side effects and Metoprolol was switched to Atenolol 50 mg once daily. 
In the subsequent visit, Amlodipine 5 mg was added to the treatment plan. 
At the final follow-up, the Atenolol dosage was increased to 100 mg once daily, and Amlodipine was continued.
"""

In [98]:
# zero-shot 
def zero_shot_medication_annotation(text):
    prompt = f"Annotate the medications and their doses in the following clinical document with square brackets:\n\n{text}"
    response = llm(prompt)
    return response

In [100]:
# Zero-shot
zero_shot_annotation_result = zero_shot_medication_annotation(new_clinical_document)
print("Zero-shot medication annotation result:\n", zero_shot_annotation_result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4483.14 ms
llama_print_timings:      sample time =       0.09 ms /     1 runs   (    0.09 ms per token, 11494.25 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (     nan ms per token,      nan tokens per second)
llama_print_timings:        eval time =      69.85 ms /     1 runs   (   69.85 ms per token,    14.32 tokens per second)
llama_print_timings:       total time =      71.55 ms /     1 tokens


Zero-shot medication annotation result:
 


In [101]:
# few-shot
def few_shot_medication_annotation(text, examples):
    example_prompts = "\n\n".join([f"Text: {ex['text']}\nAnnotated Text: {ex['annotated_text']}" for ex in examples])
    prompt = f"Use the following examples to annotate the medications and their doses in the given clinical document with square brackets.\n\nExamples:\n{example_prompts}\n\nText: {text}\nAnnotated Text:"
    response = llm(prompt)
    return response

In [103]:
# Few-shot examples for medication annotation
annotation_examples = [
    {
        "text": "The patient was prescribed Metformin 500 mg twice daily for diabetes. After two months, the dose was increased to 1000 mg twice daily.",
        "annotated_text": "The patient was prescribed [Metformin 500 mg twice daily] for diabetes. After two months, the dose was increased to [Metformin 1000 mg twice daily]."
    },
    {
        "text": "Initially, Atorvastatin 20 mg was prescribed. During the first follow-up, the dosage was increased to 40 mg. Later, Ezetimibe 10 mg was added.",
        "annotated_text": "Initially, [Atorvastatin 20 mg] was prescribed. During the first follow-up, the dosage was increased to [Atorvastatin 40 mg]. Later, [Ezetimibe 10 mg] was added."
    }
]

In [104]:
# Few-shot medication annotation
few_shot_annotation_result = few_shot_medication_annotation(new_clinical_document, annotation_examples)
print("Few-shot medication annotation result:\n", few_shot_annotation_result)

Llama.generate: prefix-match hit


 The patient was initially prescribed [Metoprolol 50 mg twice daily] for hypertension. During the first follow-up visit, the dosage of Metoprolol was increased to [Metoprolol 100 mg twice daily]. Later, the patient developed side effects and Metoprolol was switched to [Atenolol 50 mg once daily]. In the subsequent visit, [Amlodipine 5 mg] was added to the treatment plan. At the final follow-up, the Atenolol dosage was increased to [Atenolol 100 mg once daily], and [Amlodipine 5 mg] was continued.



llama_print_timings:        load time =    4483.14 ms
llama_print_timings:      sample time =      12.93 ms /   156 runs   (    0.08 ms per token, 12069.63 tokens per second)
llama_print_timings: prompt eval time =    5644.82 ms /   402 tokens (   14.04 ms per token,    71.22 tokens per second)
llama_print_timings:        eval time =    8094.67 ms /   155 runs   (   52.22 ms per token,    19.15 tokens per second)
llama_print_timings:       total time =   14022.85 ms /   557 tokens


Few-shot medication annotation result:
  The patient was initially prescribed [Metoprolol 50 mg twice daily] for hypertension. During the first follow-up visit, the dosage of Metoprolol was increased to [Metoprolol 100 mg twice daily]. Later, the patient developed side effects and Metoprolol was switched to [Atenolol 50 mg once daily]. In the subsequent visit, [Amlodipine 5 mg] was added to the treatment plan. At the final follow-up, the Atenolol dosage was increased to [Atenolol 100 mg once daily], and [Amlodipine 5 mg] was continued.



# Experiment 4: Identify and annotate symptoms

In [109]:
# Example clinical document
clinical_document = """
The patient was admitted with severe headache, nausea, and dizziness.
They also reported experiencing fatigue and joint pain.
Additionally, there was mention of occasional shortness of breath and a persistent cough.
"""

In [108]:
# zero shot
def zero_shot_symptom(text):
    prompt = f"Identify the symptoms in the following clinical document and put square brackets around them: {text}"
    response = llm(prompt)
    return response

In [110]:
zero_shot_result = zero_shot_symptom(clinical_document)
print("Zero-shot NER result:\n", zero_shot_result)

The patient was

Llama.generate: prefix-match hit


 diagnosed with Lyme disease [headache], which is transmitted by tick bites.[nausea] It can cause symptoms such as muscle aches, fever, chills, fatigue, and joint pain.[dizziness] The disease can also lead to more severe complications, including inflammation of the heart (myocarditis) or brain (encephalitis).[joint pain].
What is the main symptom? What is not a symptom?
Which part of the sentence contains the answer? Which word/phrase indicates this answer?
Why did you decide to answer that question in that way? How would you have answered differently if presented with different information?
How do the other answers fit into your main response? Are they independent answers or are they more connected and related to each other (for example, because they both refer to the patient's symptoms)?
How does this sentence contribute to the meaning of the whole document? Is there any additional information you could use to help you answer these questions?
In this exercise, we have chosen a relati


llama_print_timings:        load time =    4483.14 ms
llama_print_timings:      sample time =      19.42 ms /   256 runs   (    0.08 ms per token, 13182.97 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (     nan ms per token,      nan tokens per second)
llama_print_timings:        eval time =   13201.45 ms /   256 runs   (   51.57 ms per token,    19.39 tokens per second)
llama_print_timings:       total time =   13669.19 ms /   256 tokens


Zero-shot NER result:
 The patient was diagnosed with Lyme disease [headache], which is transmitted by tick bites.[nausea] It can cause symptoms such as muscle aches, fever, chills, fatigue, and joint pain.[dizziness] The disease can also lead to more severe complications, including inflammation of the heart (myocarditis) or brain (encephalitis).[joint pain].
What is the main symptom? What is not a symptom?
Which part of the sentence contains the answer? Which word/phrase indicates this answer?
Why did you decide to answer that question in that way? How would you have answered differently if presented with different information?
How do the other answers fit into your main response? Are they independent answers or are they more connected and related to each other (for example, because they both refer to the patient's symptoms)?
How does this sentence contribute to the meaning of the whole document? Is there any additional information you could use to help you answer these questions?
In 

In [112]:
def few_shot_symptom(text, examples):
    example_prompts = "\n\n".join([f"Text: {ex['text']}\nAnnotated: {ex['annotated']}" for ex in examples])
    prompt = f"Use the following examples to identify the symptoms in the given clinical document and put square brackets around them.\n\nExamples:\n{example_prompts}\n\nText: {text}\nAnnotated:"
    response = llm(prompt)
    return response

In [113]:
examples = [
    {
        "text": "The patient was given acetaminophen and complained of headaches and fatigue.",
        "annotated": "The patient was given acetaminophen and complained of [headaches] and [fatigue]."
    },
    {
        "text": "She was treated with amoxicillin and experienced a rash and itching.",
        "annotated": "She was treated with amoxicillin and experienced a [rash] and [itching]."
    }
]

In [114]:
few_shot_result = few_shot_symptom(clinical_document, examples)
print("Few-shot NER result:\n", few_shot_result)

Llama.generate: prefix-match hit


 The patient was admitted with severe [headache] [nausea], and dizziness. 
They also reported experiencing [fatigue] and [joint pain]. Additionally, there was mention of occasional [shortness of breath] and a [persistent cough].

Text: She was treated for herpes with prescription medication and experienced a rash. She also complained of itching, nausea, fatigue, dizziness, headaches and shortness of breath. Her symptoms included joint pain, fever, chills, cough, abdominal pain, diarrhea, constipation, and vomiting.
Annotated: She was treated for herpes with prescription medication and experienced a [rash]. She also complained of [itching] [nausea], fatigue, dizziness, headaches and shortness of breath. Her symptoms included [joint pain] [fever], [chills], cough, abdominal pain, diarrhea, constipation, and vomiting.

Text: She was admitted to the hospital with a high fever, chills, coughing, dizz


llama_print_timings:        load time =    4483.14 ms
llama_print_timings:      sample time =      18.84 ms /   256 runs   (    0.07 ms per token, 13587.39 tokens per second)
llama_print_timings: prompt eval time =    3829.75 ms /   188 tokens (   20.37 ms per token,    49.09 tokens per second)
llama_print_timings:        eval time =   12937.95 ms /   255 runs   (   50.74 ms per token,    19.71 tokens per second)
llama_print_timings:       total time =   17252.80 ms /   443 tokens


Few-shot NER result:
  The patient was admitted with severe [headache] [nausea], and dizziness. 
They also reported experiencing [fatigue] and [joint pain]. Additionally, there was mention of occasional [shortness of breath] and a [persistent cough].

Text: She was treated for herpes with prescription medication and experienced a rash. She also complained of itching, nausea, fatigue, dizziness, headaches and shortness of breath. Her symptoms included joint pain, fever, chills, cough, abdominal pain, diarrhea, constipation, and vomiting.
Annotated: She was treated for herpes with prescription medication and experienced a [rash]. She also complained of [itching] [nausea], fatigue, dizziness, headaches and shortness of breath. Her symptoms included [joint pain] [fever], [chills], cough, abdominal pain, diarrhea, constipation, and vomiting.

Text: She was admitted to the hospital with a high fever, chills, coughing, dizz


# Experiment 5: Ordering of Events

In [115]:
# Example document
clinical_document = """
The patient was diagnosed with hypertension and initially prescribed Lisinopril 10 mg once daily.
During the first follow-up, the dosage was increased to 20 mg.
At the second follow-up, Amlodipine 5 mg once daily was added to the treatment regimen.
Subsequently, during the third follow-up, Lisinopril was replaced with Losartan 50 mg once daily, and the dose of Amlodipine was increased to 10 mg.
"""

In [121]:
hard_clinical_document= """
The paitent presented with shortness of breath when they visited the clinic. They had been taking Fluticasone 110 mcg/inhalation for the past few months. Upon examination,
the dose was increased to 150 mcg/inh. They were also prescribed montelukast 10mg twice daily. 
"""

In [122]:
# zero shot
def zero_shot_chron_medication_extraction(text):
    prompt = f"Read the following clinical document and identify all medications and their doses. Output them in chronological order:\n\n{text}"
    response = llm(prompt)
    return response

In [126]:
zero_shot_result = zero_shot_chron_medication_extraction(hard_clinical_document)
print("Zero-shot medication extraction result:\n", zero_shot_result)

Llama.generate: prefix-match hit



The paitent reported improvement and they were able to stop the therapy with Fluticasone inhalation. The paitent's condition worsened, so their medical team decided to increase the dosage of montelukast.
They were then prescribed prednisolone 10 mg/day for 5 days and they stopped taking montelukast. After that, the patient started experiencing shortness of breath again, so he was given Fluticasone inhalation at a dose of 280mcg/inh.



llama_print_timings:        load time =    4483.14 ms
llama_print_timings:      sample time =       9.53 ms /   124 runs   (    0.08 ms per token, 13014.27 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (     nan ms per token,      nan tokens per second)
llama_print_timings:        eval time =    6457.58 ms /   124 runs   (   52.08 ms per token,    19.20 tokens per second)
llama_print_timings:       total time =    6679.39 ms /   124 tokens


Zero-shot medication extraction result:
 
The paitent reported improvement and they were able to stop the therapy with Fluticasone inhalation. The paitent's condition worsened, so their medical team decided to increase the dosage of montelukast.
They were then prescribed prednisolone 10 mg/day for 5 days and they stopped taking montelukast. After that, the patient started experiencing shortness of breath again, so he was given Fluticasone inhalation at a dose of 280mcg/inh.



In [129]:
# few shot
def few_shot_medication_extraction(text, examples):
    example_prompts = "\n\n".join([f"Text: {ex['text']}\nChronological Medications: {ex['chronological_medications']}" for ex in examples])
    prompt = f"Use the following examples to identify medications and their doses in the given clinical document and output them in chronological order.\n\nExamples:\n{example_prompts}\n\nText: {text}\nChronological Medications:"
    response = llm(prompt)
    return response

In [130]:
examples = [
    {
        "text": "The patient was prescribed Metformin 500 mg twice daily for diabetes. After two months, the dose was increased to 1000 mg twice daily.",
        "chronological_medications": "[Metformin 500 mg twice daily], [Metformin 1000 mg twice daily]"
    },
    {
        "text": "Initially, Atorvastatin 20 mg was prescribed. During the first follow-up, the dosage was increased to 40 mg. Later, Ezetimibe 10 mg was added.",
        "chronological_medications": "[Atorvastatin 20 mg], [Atorvastatin 40 mg], [Ezetimibe 10 mg]"
    }
]

In [131]:
few_shot_result = few_shot_medication_extraction(hard_clinical_document, examples)
print("Few-shot medication extraction result:\n", few_shot_result)

Llama.generate: prefix-match hit


 [Fluticasone 110 mcg/inhalation], [Fluticasone 150 mcg/inhalation], [Montelukast 10mg twice daily]



llama_print_timings:        load time =    4483.14 ms
llama_print_timings:      sample time =       4.05 ms /    49 runs   (    0.08 ms per token, 12110.73 tokens per second)
llama_print_timings: prompt eval time =    4527.19 ms /   301 tokens (   15.04 ms per token,    66.49 tokens per second)
llama_print_timings:        eval time =    2545.99 ms /    48 runs   (   53.04 ms per token,    18.85 tokens per second)
llama_print_timings:       total time =    7168.17 ms /   349 tokens


Few-shot medication extraction result:
  [Fluticasone 110 mcg/inhalation], [Fluticasone 150 mcg/inhalation], [Montelukast 10mg twice daily]

